In [1]:
from spinesUtils import *
import pandas as pd

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

X, y = make_classification(n_samples=10000, n_features=40, random_state=0)

df = pd.DataFrame(X, columns=[str(i) for i in range(X.shape[1])])

x_cols = df.columns.tolist()
df['y'] = y

train, test = train_test_split_bigdata_df(df, x_cols=x_cols, y_col='y', return_valid=False)
print(train.shape, test.shape)

(8000, 41) (2000, 41)


In [4]:
from lightgbm import LGBMClassifier

estimator = LGBMClassifier(random_state=0)
fe = TreeSequentialFeatureSelector(estimator, metrics_name='f1',
    forward=True,
    floating=True,
    log_file_path='feature_selection.log',
    best_features_save_path='best_feature.txt', verbose=True)

In [5]:
fe.fit(train[x_cols], train['y'])

22:00:59 2023-09-19 - file feature_selection.log not exists, will be created.
22:00:59 2023-09-19 - metrics: f1
22:00:59 2023-09-19 - excluded columns using the variation ratio: []
22:00:59 2023-09-19 - using zero variance: []
22:00:59 2023-09-19 - The initial full model was successfully obtained.
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
22:01:01 2023-09-19 - [1/39]- feas_num 2 - # best loop,  f1 score is: 0.923984, current part of dataset best score is: 0.923984
22:01:01 2023-09-19 - [2/39]- feas_num 3 - patience: 1,  f1 score is: 0.920522, current part of dataset best score is: 0.923984
22:01:01 2023-09-19 - [3/39]- feas_num 4 - patience: 2,  f1 score is: 0.919976, current part of dataset best score is: 0.923984
22:01:01 2023-09-19 - [4/39]- feas_num 5 - patience: 3,  f1 score is: 0.918757, current part of dataset best score is: 0.923984
22:01:02 2023-09-19 - [5/39]- feas_num 6 - patience: 4,  f1 score is: 0.91866, current part

In [7]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [8]:
estimator = LGBMClassifier(random_state=0)
sfs = SFS(estimator, k_features='best', floating=True, n_jobs=10, scoring='f1', verbose=True)

In [9]:
sfs.fit(train[x_cols], train['y'])

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  40 out of  40 | elapsed:    3.0s finished
Features: 1/40[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  39 out of  39 | elapsed:    1.2s finished
Features: 2/40[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  38 out of  38 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    1.4s finished
Features: 3/40[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  37 out of  37 | elapsed:    2.8s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.4s finis

SequentialFeatureSelector(estimator=LGBMClassifier(random_state=0),
                          floating=True, k_features=(1, 40), n_jobs=10,
                          scoring='f1', verbose=True)

In [10]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(sfs.transform(train[x_cols]), train['y'])

y_pred = estimator.predict(sfs.transform(test[x_cols]))

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9167072576716999


In [8]:
fe.transform(train)

(             0        21        19         9         4        36        38  \
 0     0.580775 -1.095426 -0.723644 -1.337362 -0.380448 -2.151429  0.839975   
 1     0.716031  0.235011  0.213118  0.752512 -1.324425  0.436652 -0.318493   
 2     1.434106 -1.001386 -0.599345 -0.722305 -1.750317  2.070805  1.749137   
 3     1.802176 -1.083761 -0.631431 -0.643201 -0.074499 -0.879112 -0.314738   
 4     0.972077 -1.088003 -0.691532 -1.109390 -0.563920 -0.897269 -0.552134   
 ...        ...       ...       ...       ...       ...       ...       ...   
 7995  1.981625 -0.846985 -0.454021 -0.185212 -0.756068 -0.669432 -0.020739   
 7996 -0.589596 -0.473891 -0.370939 -1.044270 -0.126329 -1.762959  0.030956   
 7997  0.299612 -0.324163 -0.205350 -0.325004  0.638018  1.945399 -0.047441   
 7998 -0.132921  0.551577  0.375356  0.761749 -1.418795  1.021512 -0.562354   
 7999  0.231037 -0.516066 -0.343846 -0.653623 -1.431277  0.165294 -1.089224   
 
              8        23  
 0    -0.375823 -1.146

In [6]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(train[fe.best_cols_], train['y'])

y_pred = estimator.predict(test[fe.best_cols_])

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9090019569471623


In [26]:
estimator = LGBMClassifier(random_state=0)

estimator.fit(train[x_cols], train['y'])

y_pred = estimator.predict(test[x_cols])

from sklearn.metrics import f1_score

print(f1_score(test['y'], y_pred))

0.9137847053093036


In [23]:
sfs.transform(train[x_cols]).shape, len(fe.cols)

((8000, 33), 32)

In [6]:
fe.best_cols_

[0,
 21,
 19,
 9,
 4,
 36,
 23,
 32,
 37,
 1,
 27,
 3,
 38,
 22,
 8,
 11,
 12,
 18,
 16,
 2,
 20,
 6,
 5,
 30,
 33,
 39,
 34,
 35,
 31,
 28,
 7,
 10,
 15]

In [ ]:
[0, 21, 32, 31, 10]